## Imports

In [4]:
# import NALU files
import sys
sys.path.insert(0, 'C:/Users/Ammar/Documents/CV LAB internship/Sproj/NALU_files')
from nalu import NALU
from nac import NAC
# import Keras items
from keras.models import Model
from keras.layers import Dense,Lambda,Concatenate,Input,Activation,Layer,Dropout
from keras import backend as K
from keras.utils import to_categorical,plot_model
from keras.initializers import constant

# import other libs

import numpy as np
# from itertools import combinations_with_replacement,product
from sklearn.model_selection import train_test_split
from tensorflow import floor as tf_floor

## Parameters

In [18]:
model_name = 'circuit-mult'
EPOCHS = 2000
BATCH_SIZE = 4
num_digits = 2
range_min = 0
range_max = 99

## Generate Dataset

In [6]:
def sep(x,num_digits):
    x = np.array([int(i) for i in str(x)])
    while (x.shape[0] < num_digits):
        
        x = np.insert(x,0,0)
    return x

In [19]:
x_train_a = np.random.randint(low = range_min,high = range_max,size =100)
x_train_b = np.random.randint(low = range_min,high = range_max,size =100)

x_train = np.empty((100,2,num_digits),dtype = np.int32)

y_train_a = x_train_a * x_train_b
y_train = np.empty((100,num_digits*2),dtype = np.int32)

for i in range(100):
    x_train[i,0] = sep(x_train_a[i],num_digits)
    
    x_train[i,1] = sep(x_train_b[i],num_digits)  
    y_train[i] = sep(y_train_a[i],num_digits*2)
    
print('\t',x_train.shape)
print('\t',y_train.shape)

	 (100, 2, 2)
	 (100, 4)


## Model

In [8]:
def net(inputs):
    input_data = Input(shape = inputs)
    out = Dense(30,activation = 'relu')(input_data)
    out = Dense(82,activation='softmax')(out)
	
    
    return Model(inputs = input_data,outputs = out)


In [15]:
model_mul = net((2,))
model_mul.trainable = False

model_mul.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics = ['accuracy'])
model_mul.summary()
model_mul.load_weights('mul_0-9_classifier.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 30)                90        
_________________________________________________________________
dense_14 (Dense)             (None, 82)                2542      
Total params: 2,632
Trainable params: 0
Non-trainable params: 2,632
_________________________________________________________________


In [10]:
def separator(inputs):
    input_data = Input(shape = inputs)
    # addition = Dense(1,kernel_initializer='ones',use_bias=False)(input_data)
    carry = Dense(1,use_bias=False,kernel_initializer=constant(value=0.1))(input_data)
    carry = Lambda(lambda x: tf_floor(x))(carry)
    
    ones_digit = Dense(1,use_bias=False,kernel_initializer=constant(value=10))(carry)
    temp = Concatenate()([ones_digit,input_data])
    ones_digit = Dense(1,use_bias=False)(temp)
    
    out = Concatenate()([carry,ones_digit])
    return Model(inputs = input_data,outputs = out)

In [11]:
model_separator = separator((1,))
model_separator.layers[5].set_weights([np.array(([-1],[1]))])
model_separator.trainable = False
model_separator.compile(loss = 'MSE',optimizer = 'Adam',metrics = ['accuracy'])
# model_separator.summary()

In [12]:
def net_mult_2(inputs):
    input_data = Input(shape = inputs)
    
    a1 = Lambda(lambda x: x[:,0,:1])(input_data)
    a2 = Lambda(lambda x: x[:,0,1:])(input_data)
    b1 = Lambda(lambda x: x[:,1,:1])(input_data)
    b2 = Lambda(lambda x: x[:,1,1:])(input_data)
    
    a1b1 = Concatenate()([a1,b1])
    a1b2 = Concatenate()([a1,b2])
    a2b1 = Concatenate()([a2,b1])
    a2b2 = Concatenate()([a2,b2])
    
    # first row
    a2b2 = model_mul(a2b2)
    a1b2 = model_mul(a1b2)
    a1b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b2)
    a2b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b2)
    sep1 = model_separator(a2b2)
    
    temp = Lambda(lambda x: x[:,0:1])(sep1)
    concat = Concatenate()([temp,a1b2])
    
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep2 = model_separator(temp)
    
    # second row
    a1b1 = model_mul(a1b1)
    a2b1 = model_mul(a2b1)
    a1b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b1)
    a2b1 = Lambda(lambda x:K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b1)
    
    sep3 = model_separator(a2b1)
    temp = Lambda(lambda x: x[:,0:1])(sep3)
    concat = Concatenate()([temp,a1b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep4 = model_separator(temp)
    
    # final row
    temp_a = Lambda(lambda x: x[:,1:2])(sep2)
    temp_b = Lambda(lambda x: x[:,1:2])(sep3)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep5 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep2)
    temp_b = Lambda(lambda x: x[:,1:2])(sep4)
    temp_c = Lambda(lambda x: x[:,0:1])(sep5)
    concat = Concatenate()([temp_a,temp_b,temp_c])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep6 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep4)
    temp_b = Lambda(lambda x: x[:,0:1])(sep6)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep7 = model_separator(temp)
    
    # answer
    temp_a = Lambda(lambda x: x[:,1:2])(sep1)
    temp_b = Lambda(lambda x: x[:,1:2])(sep5)
    temp_c = Lambda(lambda x: x[:,1:2])(sep6)
    temp_d = Lambda(lambda x: x[:,1:2])(sep7)
    
    concat = Concatenate()([temp_d,temp_c,temp_b,temp_a])
    return Model(inputs = input_data,outputs = concat)

In [92]:
def net_mult_3(inputs):
    input_data = Input(shape = inputs)
    
    a1 = Lambda(lambda x: x[:,0,:1])(input_data)
    a2 = Lambda(lambda x: x[:,0,1:2])(input_data)
    a3 = Lambda(lambda x: x[:,0,2:])(input_data)
    b1 = Lambda(lambda x: x[:,1,:1])(input_data)
    b2 = Lambda(lambda x: x[:,1,1:2])(input_data)
    b3 = Lambda(lambda x: x[:,1,2:])(input_data)
    
    a1b1 = Concatenate()([a1,b1])
    a1b2 = Concatenate()([a1,b2])
    a1b3 = Concatenate()([a1,b3])
    a2b1 = Concatenate()([a2,b1])
    a2b2 = Concatenate()([a2,b2])
    a2b3 = Concatenate()([a2,b3])
    a3b1 = Concatenate()([a3,b1])
    a3b2 = Concatenate()([a3,b2])
    a3b3 = Concatenate()([a3,b3])
    
    # first row
    a3b3 = model_mul(a3b3)
    a2b3 = model_mul(a2b3)
    a1b3 = model_mul(a1b3)

    a3b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b3)
    a2b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b3)
    a1b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b3)
    sep1 = model_separator(a3b3)
    
    temp = Lambda(lambda x: x[:,0:1])(sep1)
    concat = Concatenate()([temp,a2b3])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep2 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep2)
    concat = Concatenate()([temp,a1b3])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep3 = model_separator(temp)
    
    # second row
    a3b2 = model_mul(a3b2)
    a2b2 = model_mul(a2b2)
    a1b2 = model_mul(a1b2)

    a3b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b2)
    a2b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b2)
    a1b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b2)
    sep4 = model_separator(a3b2)
    
    temp = Lambda(lambda x: x[:,0:1])(sep4)
    concat = Concatenate()([temp,a2b2])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep5 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep5)
    concat = Concatenate()([temp,a1b2])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep6 = model_separator(temp)
    
    # third row
    a3b1 = model_mul(a3b1)
    a2b1 = model_mul(a2b1)
    a1b1 = model_mul(a1b1)

    a3b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b1)
    a2b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b1)
    a1b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b1)
    sep7 = model_separator(a3b1)
    
    temp = Lambda(lambda x: x[:,0:1])(sep7)
    concat = Concatenate()([temp,a2b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep8 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep8)
    concat = Concatenate()([temp,a1b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep9 = model_separator(temp)
    
    # final row
    temp_a = Lambda(lambda x: x[:,1:2])(sep2)
    temp_b = Lambda(lambda x: x[:,1:2])(sep4)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep10 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,1:2])(sep3)
    temp_b = Lambda(lambda x: x[:,1:2])(sep5)
    temp_c = Lambda(lambda x: x[:,1:2])(sep7)
    temp_d = Lambda(lambda x: x[:,0:1])(sep10)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep11 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep3)
    temp_b = Lambda(lambda x: x[:,1:2])(sep6)
    temp_c = Lambda(lambda x: x[:,1:2])(sep8)
    temp_d = Lambda(lambda x: x[:,0:1])(sep11)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep12 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep6)
    temp_b = Lambda(lambda x: x[:,1:2])(sep9)
    temp_c = Lambda(lambda x: x[:,0:1])(sep12)
    concat = Concatenate()([temp_a,temp_b,temp_c])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep13 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep9)
    temp_b = Lambda(lambda x: x[:,0:1])(sep13)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep14 = model_separator(temp)
    
    # answer
    temp_a = Lambda(lambda x: x[:,1:2])(sep1)
    temp_b = Lambda(lambda x: x[:,1:2])(sep10)
    temp_c = Lambda(lambda x: x[:,1:2])(sep11)
    temp_d = Lambda(lambda x: x[:,1:2])(sep12)
    temp_e = Lambda(lambda x: x[:,1:2])(sep13)
    temp_f = Lambda(lambda x: x[:,1:2])(sep14)

    concat = Concatenate()([temp_f,temp_e,temp_d,temp_c,temp_b,temp_a])
    return Model(inputs = input_data,outputs = concat)

In [85]:
def net_mult_4(inputs):
    input_data = Input(shape = inputs)
    
    a1 = Lambda(lambda x: x[:,0,0:1])(input_data)
    a2 = Lambda(lambda x: x[:,0,1:2])(input_data)
    a3 = Lambda(lambda x: x[:,0,2:3])(input_data)
    a4 = Lambda(lambda x: x[:,0,3:4])(input_data)
    b1 = Lambda(lambda x: x[:,1,0:1])(input_data)
    b2 = Lambda(lambda x: x[:,1,1:2])(input_data)
    b3 = Lambda(lambda x: x[:,1,2:3])(input_data)
    b4 = Lambda(lambda x: x[:,1,3:4])(input_data)

    a1b1 = Concatenate()([a1,b1])
    a1b2 = Concatenate()([a1,b2])
    a1b3 = Concatenate()([a1,b3])
    a1b4 = Concatenate()([a1,b4])
    a2b1 = Concatenate()([a2,b1])
    a2b2 = Concatenate()([a2,b2])
    a2b3 = Concatenate()([a2,b3])
    a2b4 = Concatenate()([a2,b4])
    a3b1 = Concatenate()([a3,b1])
    a3b2 = Concatenate()([a3,b2])
    a3b3 = Concatenate()([a3,b3])
    a3b4 = Concatenate()([a3,b4])
    a4b1 = Concatenate()([a4,b1])
    a4b2 = Concatenate()([a4,b2])
    a4b3 = Concatenate()([a4,b3])
    a4b4 = Concatenate()([a4,b4])
    
    # first row
    a4b4 = model_mul(a4b4)
    a3b4 = model_mul(a3b4)
    a2b4 = model_mul(a2b4)
    a1b4 = model_mul(a1b4)

    a4b4 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a4b4)
    a3b4 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b4)
    a2b4 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b4)
    a1b4 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b4)
    
    sep1 = model_separator(a4b4)
    
    temp = Lambda(lambda x: x[:,0:1])(sep1)
    concat = Concatenate()([temp,a3b4])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep2 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep2)
    concat = Concatenate()([temp,a2b4])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep3 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep3)
    concat = Concatenate()([temp,a1b4])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep4 = model_separator(temp)
    
    # second row
    a4b3 = model_mul(a4b3)
    a3b3 = model_mul(a3b3)
    a2b3 = model_mul(a2b3)
    a1b3 = model_mul(a1b3)

    a4b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a4b3)
    a3b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b3)
    a2b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b3)
    a1b3 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b3)
    sep5 = model_separator(a4b3)
    
    temp = Lambda(lambda x: x[:,0:1])(sep5)
    concat = Concatenate()([temp,a3b3])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep6 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep6)
    concat = Concatenate()([temp,a2b3])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep7 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep7)
    concat = Concatenate()([temp,a1b3])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep8 = model_separator(temp)
    
    # third row
    a4b2 = model_mul(a4b2)
    a3b2 = model_mul(a3b2)
    a2b2 = model_mul(a2b2)
    a1b2 = model_mul(a1b2)

    a4b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a4b2)
    a3b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b2)
    a2b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b2)
    a1b2 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b2)
    sep9 = model_separator(a4b2)
    
    temp = Lambda(lambda x: x[:,0:1])(sep9)
    concat = Concatenate()([temp,a3b2])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep10 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep10)
    concat = Concatenate()([temp,a2b2])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep11 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep11)
    concat = Concatenate()([temp,a1b2])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep12 = model_separator(temp)
    
    # fourth row
    
    a4b1 = model_mul(a4b1)
    a3b1 = model_mul(a3b1)
    a2b1 = model_mul(a2b1)
    a1b1 = model_mul(a1b1)

    a4b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a4b1)
    a3b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a3b1)
    a2b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a2b1)
    a1b1 = Lambda(lambda x: K.expand_dims(K.cast(K.argmax(x),dtype ='float32')))(a1b1)
    sep13 = model_separator(a4b1)
    
    temp = Lambda(lambda x: x[:,0:1])(sep13)
    concat = Concatenate()([temp,a3b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep14 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep14)
    concat = Concatenate()([temp,a2b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep15 = model_separator(temp)
    
    temp = Lambda(lambda x: x[:,0:1])(sep15)
    concat = Concatenate()([temp,a1b1])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep16 = model_separator(temp)
    
    # final row
    temp_a = Lambda(lambda x: x[:,1:2])(sep2)
    temp_b = Lambda(lambda x: x[:,1:2])(sep5)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep17 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,1:2])(sep3)
    temp_b = Lambda(lambda x: x[:,1:2])(sep6)
    temp_c = Lambda(lambda x: x[:,1:2])(sep9)
    temp_d = Lambda(lambda x: x[:,0:1])(sep17)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep18 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,1:2])(sep4)
    temp_b = Lambda(lambda x: x[:,1:2])(sep7)
    temp_c = Lambda(lambda x: x[:,1:2])(sep10)
    temp_d = Lambda(lambda x: x[:,1:2])(sep13)
    temp_e = Lambda(lambda x: x[:,0:1])(sep18)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d,temp_e])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep19 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep4)
    temp_b = Lambda(lambda x: x[:,1:2])(sep8)
    temp_c = Lambda(lambda x: x[:,1:2])(sep11)
    temp_d = Lambda(lambda x: x[:,1:2])(sep14)
    temp_e = Lambda(lambda x: x[:,0:1])(sep19)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d,temp_e])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep20 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep8)
    temp_b = Lambda(lambda x: x[:,1:2])(sep12)
    temp_c = Lambda(lambda x: x[:,1:2])(sep15)
    temp_d = Lambda(lambda x: x[:,0:1])(sep20)
    concat = Concatenate()([temp_a,temp_b,temp_c,temp_d])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep21 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep12)
    temp_b = Lambda(lambda x: x[:,1:2])(sep16)
    temp_c = Lambda(lambda x: x[:,0:1])(sep21)
    concat = Concatenate()([temp_a,temp_b,temp_c])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep22 = model_separator(temp)
    
    temp_a = Lambda(lambda x: x[:,0:1])(sep16)
    temp_b = Lambda(lambda x: x[:,0:1])(sep22)
    concat = Concatenate()([temp_a,temp_b])
    temp = Dense(1,use_bias=False,kernel_initializer='ones')(concat)
    sep23 = model_separator(temp)
    
    # answer
    temp_a = Lambda(lambda x: x[:,1:2])(sep1)
    temp_b = Lambda(lambda x: x[:,1:2])(sep17)
    temp_c = Lambda(lambda x: x[:,1:2])(sep18)
    temp_d = Lambda(lambda x: x[:,1:2])(sep19)
    temp_e = Lambda(lambda x: x[:,1:2])(sep20)
    temp_f = Lambda(lambda x: x[:,1:2])(sep21)
    temp_g = Lambda(lambda x: x[:,1:2])(sep22)
    temp_h = Lambda(lambda x: x[:,1:2])(sep23)
    
    concat = Concatenate()([temp_h,temp_g,temp_f,temp_e,temp_d,temp_c,temp_b,temp_a])

    return Model(inputs = input_data,outputs = concat)

+ #### Model Initilization And Details

In [13]:
model = net_mult_2((2,num_digits))
model.trainable=False
model.compile(loss = 'MSE',optimizer = 'Adam',metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2, 2)         0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1)            0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 1)            0           input_3[0][0]                    
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 2)            0           lambda_3[0][0]                   
                                                                 lambda_4[0][0]                   
__________

+ #### Test Model

In [13]:
out = model.predict(x_train)
for i in range(100):
    print ('\n \t',x_train[i,0],' * ',x_train[i,1],' = ', y_train[i])
    print('\t predicted: ',out[i])


 	 [9 5]  *  [9 2]  =  [8 7 4 0]
	 predicted:  [8. 7. 4. 0.]

 	 [5 1]  *  [0 7]  =  [0 3 5 7]
	 predicted:  [0. 3. 5. 7.]

 	 [1 9]  *  [3 4]  =  [0 6 4 6]
	 predicted:  [0. 6. 4. 6.]

 	 [4 1]  *  [5 8]  =  [2 3 7 8]
	 predicted:  [2. 3. 7. 8.]

 	 [5 7]  *  [2 9]  =  [1 6 5 3]
	 predicted:  [1. 6. 5. 3.]

 	 [5 8]  *  [0 6]  =  [0 3 4 8]
	 predicted:  [0. 3. 4. 8.]

 	 [5 4]  *  [1 1]  =  [0 5 9 4]
	 predicted:  [0. 5. 9. 4.]

 	 [5 5]  *  [7 7]  =  [4 2 3 5]
	 predicted:  [4. 2. 3. 5.]

 	 [3 4]  *  [7 0]  =  [2 3 8 0]
	 predicted:  [2. 3. 8. 0.]

 	 [0 7]  *  [0 1]  =  [0 0 0 7]
	 predicted:  [0. 0. 0. 7.]

 	 [3 4]  *  [5 7]  =  [1 9 3 8]
	 predicted:  [1. 9. 3. 8.]

 	 [9 6]  *  [3 7]  =  [3 5 5 2]
	 predicted:  [3. 5. 5. 2.]

 	 [1 9]  *  [1 8]  =  [0 3 4 2]
	 predicted:  [0. 3. 4. 2.]

 	 [1 3]  *  [2 0]  =  [0 2 6 0]
	 predicted:  [0. 2. 6. 0.]

 	 [6 7]  *  [1 2]  =  [0 8 0 4]
	 predicted:  [0. 8. 0. 4.]

 	 [9 6]  *  [7 9]  =  [7 5 8 4]
	 predicted:  [7. 5. 8. 4.]

 	 [8 0

+ #### Save Model

In [14]:
plot_model(model,to_file = 'final.png',show_shapes=True)